In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os
import pickle
from pymongo import MongoClient

In [20]:
chromedriver ="/home/zhouza/chromedriver"
#chromedriver = "../../chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

In [131]:
driver = webdriver.Chrome(chromedriver)

In [22]:
base_url = 'http://cts.perseids.org'

In [23]:
def init_authors():
    driver.get('http://cts.perseids.org/read/latinLit')
    links = []
    authors_ul = driver.find_elements_by_class_name('menu')[1]
    authors = authors_ul.find_elements_by_tag_name('a')
    for author in authors:
        # remove numbers and special characters from author name
        author_name = re.sub(r'[^a-zA-Z\s]*','',author.text.strip())
        links.append({'author':author_name, 'url':author.get_attribute('href')})

    return links

In [ ]:
links = init_authors()

In [24]:
def add_works(links):
    
    for author in links: 
        author_name = author['author']
        # driver.get(author['url'])

        works_soup = BeautifulSoup(requests.get(author['url']).text,'lxml')
        works = works_soup.findAll('h2')
        for work in works:
            work_name = re.sub(r'[^a-zA-Z\s]*','',work.text.strip())

            # check if Latin text is available
            try:
                work_url = base_url + work.findNextSibling().find('a',text=['Latin','Latin ']).attrs['href']
            except:
                work_url = None

            find_idx = [idx for idx,entry in enumerate(links) if entry['author'] == author_name][0]
            # check if content key already created
            try:
                links[find_idx]['content'].append({'work':work_name, 'url':work_url})
            except:
                links[find_idx]['content'] = [{'work':work_name, 'url':work_url}]
    
    return links

In [ ]:
links = add_works(links)

In [129]:
def get_text(url):

    driver.get(url)
    try:
        text_links = driver.find_element_by_class_name('reffs').find_elements_by_tag_name('a')
        link_list = [link.get_attribute('href') for link in text_links]

        text_list = []
        for link in link_list:
            driver.get(link)
            try:
                text = driver.find_element_by_tag_name('section').text
                cleaned_text = re.sub(r'[\s\xa0]+',' ',text.strip())
            except:
                text = ''
            text_list.append(text)
        text_str = '\n'.join(text_list)
    except:
        text_str = ''
    return text_str

In [103]:
pickle.dump(links,open('pickles/links.pkl','wb'))

In [9]:
links = pickle.load(open('pickles/links.pkl','rb'))

In [84]:
client = MongoClient()
db = client.latinlit
data = db.data

In [79]:
# data = []

for author in links[-1:]:
    author_name = author['author']
    for work in author['content']:
        work_name = work['work']
        if work['url'] != None:
            text = get_text(work['url'])
            # data.append({'author':author_name, 'work':work_name, 'text':text})
            data.insert_one({'author':author_name, 'work':work_name, 'text':text})

In [85]:
data.find({}).count()

631

In [133]:
missing = list(data.find({'text':''}))

In [132]:
for doc in missing:
    author_name = doc['author']
    work_name = doc['work']
    link_idx = [idx for idx,entry in enumerate(links) if entry['author'] == author_name][0]
    work_idx = [idx for idx,entry in enumerate(links[link_idx]['content']) if entry['work'] == work_name][0]
    work_url = links[link_idx]['content'][work_idx]['url']
    if work_url != None:
        text = get_text(work_url)
        # data.append({'author':author_name, 'work':work_name, 'text':text})
        data.delete_one({'author':author_name, 'work':work_name})
        data.insert_one({'author':author_name, 'work':work_name, 'text':text})

In [135]:
scraped_data = list(data.find({}))
pickle.dump(scraped_data,open('pickles/scraped_data.pkl','wb'))

In [139]:
root_data = db.rootdata
split_data = db.splitdata

In [143]:
root_data.find({}).count()

38575

In [141]:
split_data.find({}).count()

117194